In [1]:
from data.gen_data import build_X, build_Y, build_XY
from data.gen_data_config import gen_data_config
from evaluation import score
import pandas as pd
import numpy as np
from util import plot_roi, categori_reverse
from model import build_model
from keras.optimizers import SGD, Adam, RMSprop, Adagrad
import keras
from pickle import load, dump
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
embedding_dim = 128
batch_size = 128
epochs = 300

In [3]:
tr_s, tr_e = '2001/1/1', '2018/12/31'
te_s, te_e = '2019/1/1', '2019/12/31'
feature_days = 3
result_roi = {
    'rnn': {},
    'dense': {},
}

In [4]:
with open(f'./embedding_data/rnn_{embedding_dim}', 'rb') as f:
    embed_rnn = load(f)
with open(f'./embedding_data/dense_{embedding_dim}', 'rb') as f:
    embed_dense = load(f)

In [5]:
_, tr_y, tr_result = build_XY(tr_s, tr_e, 
                                     gen_data_config['selected_features'], 
                                     feature_days)
te_x, te_y, te_result = build_XY(te_s, te_e, 
                                 gen_data_config['selected_features'], 
                                 feature_days)
# convert class vectors to binary class matrices
one_tr_y = keras.utils.to_categorical(tr_y, 3)
one_te_y = keras.utils.to_categorical(te_y, 3)

In [ ]:
for task in ['1', '2a', '2b']:
    '''rnn model'''
    tr_x = embed_rnn['data'][task]
    te_x = embed_rnn['tar'][task]
    re_tr_x = np.reshape(tr_x, (-1, 1, embedding_dim))
    re_te_x = np.reshape(te_x, (-1, 1, embedding_dim))
    
    model_rnn = build_model('rnn' ,re_tr_x.shape, one_tr_y.shape)
    
    ''' Compile model with specified loss and optimizer '''
    model_rnn.compile(loss= 'categorical_crossentropy',
                       optimizer='Adam',
                       metrics=['accuracy'])


    '''Fit models and use validation_split=0.1 '''
    history = model_rnn.fit(re_tr_x, one_tr_y,
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  verbose=0,
                                  shuffle=True,
                                  validation_split=0.1,)
    
    pred_y = model_rnn.predict(re_te_x)
    pred_z = categori_reverse(pred_y)
    result_roi['rnn'][task] = score(pred_z, te_result)
    
    
    '''dense model'''
    tr_x = embed_dense['data'][task]
    te_x = embed_dense['tar'][task]
    
    model_dense = build_model('dense' ,tr_x.shape, one_tr_y.shape)

    ''' Compile model with specified loss and optimizer '''
    model_dense.compile(loss= 'categorical_crossentropy',
                       optimizer='Adam',
                       metrics=['accuracy'])

    '''Fit models and use validation_split=0.1 '''
    history = model_dense.fit(tr_x, one_tr_y,
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  verbose=0,
                                  shuffle=True,
                                  validation_split=0.1,)
    pred_y = model_dense.predict(te_x)
    pred_z = categori_reverse(pred_y)
    result_roi['dense'][task] = score(pred_z, te_result)

#2action_day: 113, #3consecutive_loss_max: -198, #4consecutive_loss_days_max: 8, #5loss_days: 78, roi: -517, roi_max: 44, roi_min: -660
#2action_day: 77, #3consecutive_loss_max: -205, #4consecutive_loss_days_max: 10, #5loss_days: 53, roi: -240, roi_max: 162, roi_min: -326
#2action_day: 129, #3consecutive_loss_max: -280, #4consecutive_loss_days_max: 12, #5loss_days: 82, roi: 46, roi_max: 553, roi_min: -69
#2action_day: 101, #3consecutive_loss_max: -161, #4consecutive_loss_days_max: 6, #5loss_days: 66, roi: -219, roi_max: 97, roi_min: -339
#2action_day: 14, #3consecutive_loss_max: -117, #4consecutive_loss_days_max: 5, #5loss_days: 11, roi: -89, roi_max: 6, roi_min: -111


In [ ]:
roi_df_dict = {
    'rnn': {},
    'dense': {},
}
for t in ['1', '2a', '2b']:
    roi_df_dict['rnn'][t] = plot_roi(result_roi['rnn'][t], te_result)
    roi_df_dict['dense'][t] = plot_roi(result_roi['dense'][t], te_result)

In [ ]:
fig = plt.figure(figsize=(25, 35))
ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

for i, t in zip(range(1,4), ['1', '2a', '2b']):
    i = i*2
    left, right = i-1, i
    roi_df_dict['rnn'][t].plot(rot=75,ax = locals()[f'ax{left}'], fontsize=15)
    roi_df_dict['dense'][t].plot(rot=75,ax = locals()[f'ax{right}'], fontsize=15)
    locals()[f'ax{left}'].set_title(f'RNN_task_{t}', fontsize=20)
    locals()[f'ax{right}'].set_title(f'Dense_task_{t}', fontsize=20)
fig.suptitle(f'embedding dim {embedding_dim}', fontsize=30)

In [ ]:
fig.savefig(f'./result/result_{embedding_dim}.png')
file = open(f'./result/roi_df_{embedding_dim}.pkl', 'wb')
dump(roi_df_dict, file)